In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
from PIL import Image  # Импорт библиотеки для работы с изображениями
from transformers import CLIPProcessor, CLIPModel  # Импорт модели и процессора CLIP
import pandas as pd  # Импорт библиотеки для работы с данными
import seaborn as sns  # Импорт библиотеки для визуализации данных
import matplotlib.pyplot as plt  # Импорт библиотеки для построения графиков
from sklearn.metrics import confusion_matrix  # Импорт функции для построения матрицы путаницы
import torch  # Импорт библиотеки для работы с тензорами и моделями
import numpy as np  # Импорт библиотеки для работы с массивами

# 1. Загрузка предобученной модели CLIP и её процессора
model_name = "openai/clip-vit-base-patch32"  
model = CLIPModel.from_pretrained(model_name) 
processor = CLIPProcessor.from_pretrained(model_name)  

# 2. Чтение данных из CSV-файла
df = pd.read_csv('Weather-Analysis-main/weather_new_log.csv')  

# 3. Функция для преобразования числовых меток в текстовые описания
def razmetka(target):
    """Функция для преобразования числовых меток в текстовые описания."""
    if target == 0:
        return 'cloudy sky'
    elif target == 1:
        return 'rainy weather'
    elif target == 2:
        return 'sunny day'
    elif target == 3:
        return 'sunrise'

# 3. Применение функции разметки к столбцу 'target'
df['target'] = df['target'].apply(lambda x: razmetka(x))  

# 4. Создание нескольких наборов промтов для каждой метки
labels = ["cloudy sky", "rainy weather", "sunny day", "sunrise"]

prompts_set1 = [f"a photo of a {label}" for label in labels]
prompts_set2 = [f"an image showing {label}" for label in labels]
prompts_set3 = [f"a picture of {label} with typical features" for label in labels]

prompts_sets = [prompts_set1, prompts_set2, prompts_set3]

predicted = []

# 4. Предсказание меток для каждого изображения в DataFrame.

# Проход по каждой строке DataFrame
for image_source in df['image_path'].to_list():
    image_link = 'Weather-Analysis-main\\' + image_source
    # Открытие изображения
    image = Image.open(image_link)  
    all_probs = []
    # Проход по каждому набору промтов
    for prompts in prompts_sets:
        # Подготовка входных данных
        inputs = processor(text=prompts, images=image, return_tensors="pt", padding=True)  
        with torch.no_grad():
            outputs = model(**inputs)  # Получение выходных данных от модели
            logits_per_image = outputs.logits_per_image  # Извлечение логов для изображений
            
            # Вычисление вероятностей с помощью softmax
            probs = logits_per_image.softmax(dim=1).cpu().numpy()[0]
            all_probs.append(probs)
    
    # Усреднить вероятности
    avg_probs = np.mean(all_probs, axis=0)
    pred_index = avg_probs.argmax()
    predicted.append(labels[pred_index])
    
    print(f"Предсказанное значение: {labels[pred_index]} (Уверенность {avg_probs[pred_index]:.2f}) , а истенный класс {df[df['image_path'] == image_source]['target'].values[0]}")

df['predicted'] = predicted

# Построение матрицы путаницы
cm = confusion_matrix(df['target'], df['predicted'], labels=df['target'].unique())
sns.heatmap(cm, annot=True, fmt='d', xticklabels=df['target'].unique(), yticklabels=df['target'].unique())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Вывод отчёта о классификации
from sklearn.metrics import classification_report
print(classification_report(df['target'], df['predicted']))
